In [ ]:
#                                                                         # Version Python 3.11.1
import pandas as pd                                                       # Version 2.0.1
from sqlalchemy import create_engine, text                                # Version 2.0.13
from selenium import webdriver                                            # Version 4.9.1
from selenium.webdriver.chrome.service import Service as ChromeService    # Version 4.9.1
from webdriver_manager.chrome import ChromeDriverManager                  # Version 3.8.6
from selenium.webdriver.common.by import By                               # Version 4.9.1
import time
import random
import datetime
print("fertig")

In [ ]:
## SQL-Server Verbindung erstellen
# Angaben zum SQL-Server
host=""
database=""
user=""
password=""

# Erstelle eine SQL-Verbindung mit der Datenbank
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')
print("fertig")

In [ ]:
# Daten aus Rohdaten-Tabelle laden
daten_aus_sql = pd.read_sql_query(f"SELECT seite, url FROM jobs.aktuell", connection)
print("fertig")

In [ ]:
# die eingelesenen Daten aufteilen nach den Stellenbörsen
daten_stepstone=daten_aus_sql[daten_aus_sql["seite"] == 'stepstone'].reset_index().drop(columns='index')
daten_indeed=daten_aus_sql[daten_aus_sql["seite"] == 'indeed'].reset_index().drop(columns='index')
daten_monster=daten_aus_sql[daten_aus_sql["seite"] == 'monster'].reset_index().drop(columns='index')
daten_linkedin=daten_aus_sql[daten_aus_sql["seite"] == 'linkedin'].reset_index().drop(columns='index')

In [ ]:
# Öffnen des Browser´s
chrome_options = webdriver.ChromeOptions()
#Headless-Modus aktivieren
#chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
time.sleep(1)
driver.maximize_window()
print("Browser geöffnet")

In [ ]:
# Liste für die URL´s wo die Stellenazeigen icht mehr aktuell sind
seite_liste=[]
nicht_mehr_existierende_stellenanzeigen=[]
Grund_liste=[]
Datum_liste=[]
print("fertig")

In [ ]:
# suche nicht mehr vorhandenen Stellenanzeigen in "stepstone"
print(f'Stepstone hat {len(daten_stepstone)} Datensätze. Start: {datetime.datetime.now().strftime("%H:%M:%S")}')
for url in daten_stepstone.url:
    # Webseite aufrufen
    try:
        driver.get(url)
    except:
        pass
    
    ## Überprüfen der Seite nach verschiedene Kritereien
    # Kriterium: Es wird angezeigt, das die Stellenanzeige nicht mehr existiert
    try:
        if driver.find_element(By.CLASS_NAME, 'listing-content-provider-1qtvd67').text == 'Diese Stellenanzeige ist nicht mehr verfügbar.':
            seite_liste.append('stepstone')
            nicht_mehr_existierende_stellenanzeigen.append(url)
            Grund_liste.append('Stellenanzeige nicht verfügbar')
            Datum_liste.append(datetime.datetime.now())
    except:
        pass
    
    # Krieterium: Server Error 404
    try:
        if driver.find_element(By.CLASS_NAME, 'err-kxw8ad').text == 'Server error 404':
            seite_liste.append('stepstone')
            nicht_mehr_existierende_stellenanzeigen.append(url)
            Grund_liste.append('Server error 404')
            Datum_liste.append(datetime.datetime.now())
    except:
        pass
    
    # Wartezeit zum Aufbau der neuen Seite
    time.sleep(1)
print("fertig")

In [ ]:
# suche nicht mehr vorhandenen Stellenanzeigen in "indeed"
print(f'Indeed hat {len(daten_indeed)} Datensätze. Start: {datetime.datetime.now().strftime("%H:%M:%S")}')
for url in daten_indeed.url:
    # Webseite aufrufen
    try:
        driver.get(url)
    except:
        pass
        
    ## Überprüfen der Seite nach verschiedene Kritereien
    # Kriterium: Es wird angezeigt, das die Stellenanzeige nicht mehr existiert
    try:
        if driver.find_element(By.CLASS_NAME, 'css-jsuk8h').text == 'Diese Stellenanzeige ist auf Indeed abgelaufen':
            seite_liste.append('indeed')
            nicht_mehr_existierende_stellenanzeigen.append(url)
            Grund_liste.append('Stellenanzeige nicht verfügbar')
            Datum_liste.append(datetime.datetime.now())
    except:
        pass
    # Wartezeit zum Aufbau der neuen Seite
    time.sleep(random.randint(3,10))
print("fertig")

In [ ]:
# suche nicht mehr vorhandenen Stellenanzeigen in "monster"
print(f'Monster hat {len(daten_monster)} Datensätze. Start: {datetime.datetime.now().strftime("%H:%M:%S")}')
for url in daten_monster.url:
    # Webseite aufrufen
    try:
        driver.get(url)
        time.sleep(random.random(1,2))
        
        # "Webseite ist nicht erreichbar" wird angezeigt
        try:
            driver.find_element(By.XPATH, '//*[@id="reload-button"]').click() # auf Button "neu laden" klicken
        except:
            pass
    except:
        pass
        
    ## Überprüfen der Seite nach verschiedene Kritereien
    # Kriterium: Es wird angezeigt, das die Stellenanzeige nicht mehr existiert
    try:
        if driver.find_element(By.CLASS_NAME, 'message-containerstyles__MessageTitle-sc-j0jexe-6').text == 'Oh, oh! Diese Stellenausschreibung ist abgelaufen.':
            seite_liste.append('monster')
            nicht_mehr_existierende_stellenanzeigen.append(url)
            Grund_liste.append('Stellenanzeige nicht verfügbar')
            Datum_liste.append(datetime.datetime.now())
    except:
        pass
    # Wartezeit zum Aufbau der neuen Seite
    time.sleep(random.randint(3,10))
print("fertig")

In [ ]:
# suche nicht mehr vorhandenen Stellenanzeigen in "linkedin"
print(f'LinkedIn hat {len(daten_linkedin)} Datensätze. Start: {datetime.datetime.now().strftime("%H:%M:%S")}')
for url in daten_linkedin.url:
    # Webseite aufrufen
    try:
        driver.get(url)
    except:
        pass
        
    ## Überprüfen der Seite nach verschiedene Kritereien
    #Kriterium: Es wird angezeigt, das die Stellenanzeige nicht mehr existiert
    try:
        if driver.find_element(By.CLASS_NAME, 'inline-notification__text').text == 'Dieses Jobangebot ist nicht mehr verfügbar. Aber vielleicht ist eine dieser ähnlichen Stellen für Sie interessant.':
            seite_liste.append('linkedin')
            nicht_mehr_existierende_stellenanzeigen.append(url)
            Grund_liste.append('Stellenanzeige nicht verfügbar')
            Datum_liste.append(datetime.datetime.now())
    except:
        pass
    
    # Kriterium: Anmeldemaske -> wird ignoriert da evtl. als BOT erkannt
    try:
        if driver.find_element(By.CLASS_NAME, 'authwall-join-form__title').text == 'Mitglied werden':
            pass
    except:
        pass
    # Wartezeit zum Aufbau der neuen Seite
    time.sleep(random.randint(3,10))
print("fertig")

In [ ]:
# Browser schließen
driver.close()
print('Browser geschlossen')

In [ ]:
# DataFrame erzeugen
try:
    df=pd.DataFrame({"seite": seite_liste,
                    "url": nicht_mehr_existierende_stellenanzeigen,
                    "grund": Grund_liste,
                    "datum": Datum_liste   
    })

    display(df)
except ValueError as e:
    print(f"Dataframe konnte nicht erstellt werden\n{e}")

In [ ]:
df.grund.value_counts()

In [ ]:
# Rohdatenbank aktualisieren
print(f'Aktuallisiere Datenbank: - Zeit: {datetime.datetime.now().strftime("%H:%M:%S")}')
with connection.connect() as SQL_verbindung:
    for url in df.url:
        sql_befehl=text(f"UPDATE jobs.rohdaten SET storno = true WHERE url = :url;")
        SQL_verbindung.execute(sql_befehl.bindparams(url=url))
    SQL_verbindung.commit()
print(f'Aktuallisierung beendet: - Zeit: {datetime.datetime.now().strftime("%H:%M:%S")}')

# DataFrame in Datenbank speichern
print(f'Trage Daten in die LOG-Tabelle der Datenbank ein: - Zeit: {datetime.datetime.now().strftime("%H:%M:%S")}')
try:
    df.to_sql(name='log_storno', schema='jobs',
            con=connection, if_exists='append', index=False, )
    print(f'Eintragung beendet: - Zeit: {datetime.datetime.now().strftime("%H:%M:%S")}')
except NameError as e:
    print(f"Kein DataFrame vorhanden\n{e}")